In [1]:
import mmcv
from mmcv import Config
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.models import build_detector
from mmdet.apis import single_gpu_test
from mmcv.runner import load_checkpoint
import os
from mmcv.parallel import MMDataParallel
import pandas as pd
from pycocotools.coco import COCO

In [3]:
class InferMMDet:
    def __init__(self, samples_per_gpu: int =None, workers_per_gpu: int =None) -> None:
        self.cfg = Config.fromfile('/opt/ml/Git/p_stage_obj_det/mmdet/config/models/swin/swin-s_img-768_AdamW-20e.py')

        if samples_per_gpu:
            self.cfg.data.samples_per_gpu = samples_per_gpu

        if workers_per_gpu:
            self.cfg.data.workers_per_gpu = workers_per_gpu

        self.cfg.gpu_ids = [0]

        self.cfg.work_dir = '/opt/ml/Git/p_stage_obj_det/mmdet/config/work_dirs/swin-s_img-768_AdamW-20e/'

        # cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
        self.cfg.model.train_cfg = None

        # build dataset & dataloader
        dataset = build_dataset(self.cfg.data.test)
        self.data_loader = build_dataloader(
                dataset,
                samples_per_gpu=self.cfg.data.samples_per_gpu,
                workers_per_gpu=self.cfg.data.workers_per_gpu,
                dist=False,
                shuffle=False)

        # checkpoint path
        checkpoint_path = os.path.join(self.cfg.work_dir, f'epoch_3.pth')

        model = build_detector(self.cfg.model, test_cfg=self.cfg.get('test_cfg')) # build detector
        checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu') # ckpt load
        # checkpoint = load_checkpoint(model, checkpoint_path) # ckpt load
        self.checkpoint = checkpoint

        model.CLASSES = dataset.CLASSES
        model = MMDataParallel(model.cuda(), device_ids=[0])
        self.model = model

    def inference(self):
        return single_gpu_test(self.model, self.data_loader, show_score_thr=0.05) # output 계산
    
    def output2dataframe(self, output, save_file_name):
        assert output, 'output required !!'
        # submission 양식에 맞게 output 후처리
        prediction_strings = []
        file_names = []
        coco = COCO(self.cfg.data.test.ann_file)
        img_ids = coco.getImgIds()

        class_num = 10
        for i, out in enumerate(output):
            prediction_string = ''
            image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
            for j in range(class_num):
                for o in out[j]:
                    prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                        o[2]) + ' ' + str(o[3]) + ' '
                
            prediction_strings.append(prediction_string)
            file_names.append(image_info['file_name'])


        submission = pd.DataFrame()
        submission['PredictionString'] = prediction_strings
        submission['image_id'] = file_names
        self.output_df = submission
        if save_file_name:
            submission.to_csv(os.path.join(self.cfg.work_dir, save_file_name), index=None)


In [6]:
infer_model = InferMMDet()
output = infer_model.inference()

loading annotations into memory...
Done (t=0.06s)
creating index...
index created!
Use load_from_local loader
[                                 ] 1/488, 0.1 task/s, elapsed: 9s, ETA:  4619s

/opt/ml/Git/mmdetection/mmdet/core/anchor/anchor_generator.py:324: UserWarning: ``grid_anchors`` would be deprecated soon. Please use ``grid_priors`` 
  warnings.warn('``grid_anchors`` would be deprecated soon. '
/opt/ml/Git/mmdetection/mmdet/core/anchor/anchor_generator.py:361: UserWarning: ``single_level_grid_anchors`` would be deprecated soon. Please use ``single_level_grid_priors`` 
  '``single_level_grid_anchors`` would be deprecated soon. '


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 488/488, 1.8 task/s, elapsed: 279s, ETA:     0s

In [7]:
output

[[array([[3.1066937e+02, 2.7796344e+02, 7.1144672e+02, 6.9901050e+02,
          1.2209741e-01],
         [3.2265887e+02, 2.8239624e+02, 6.3545282e+02, 4.4865582e+02,
          5.8107516e-03]], dtype=float32),
  array([[3.1066937e+02, 2.7796344e+02, 7.1144672e+02, 6.9901050e+02,
          6.4105438e-03]], dtype=float32),
  array([[3.0975873e+02, 2.7802252e+02, 7.1295306e+02, 6.9955292e+02,
          9.6432722e-01],
         [3.0955188e+02, 2.7875177e+02, 7.1182117e+02, 6.9667822e+02,
          9.8968688e-03],
         [3.2667215e+02, 2.8273285e+02, 6.4942786e+02, 4.6285010e+02,
          1.1871993e-03]], dtype=float32),
  array([[3.1070245e+02, 2.7860297e+02, 7.1020349e+02, 6.9951770e+02,
          1.9805667e-01],
         [3.0947675e+02, 2.7913132e+02, 7.1200995e+02, 6.9765100e+02,
          1.4838127e-03]], dtype=float32),
  array([], shape=(0, 5), dtype=float32),
  array([[3.1070245e+02, 2.7860297e+02, 7.1020349e+02, 6.9951770e+02,
          1.8062039e-01],
         [3.0935373e+02, 2

In [ ]:
output = single_gpu_test(model, data_loader, show_score_thr=0.05) # output 계산

In [ ]:
# submission 양식에 맞게 output 후처리
prediction_strings = []
file_names = []
coco = COCO(cfg.data.test.ann_file)
img_ids = coco.getImgIds()

class_num = 10
for i, out in enumerate(output):
    prediction_string = ''
    image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
    for j in range(class_num):
        for o in out[j]:
            prediction_string += str(j) + ' ' + str(o[4]) + ' ' + str(o[0]) + ' ' + str(o[1]) + ' ' + str(
                o[2]) + ' ' + str(o[3]) + ' '
        
    prediction_strings.append(prediction_string)
    file_names.append(image_info['file_name'])


submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.work_dir, f'submission_3_msTTA_with_train.csv'), index=None)
submission.head()